# These series of functions are made to convert and handle TEM images from a dm4 format. The images are converted into numpy data sets, image data is normalized and contrast is adjusted, optional scale bars are created, and then the images are saved into the selected folder as tiff files.

# Make TIF images

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt 
import os
import ncempy.io as nc
from skimage.io import imsave
import sys
import shutil
from PIL import Image, ImageDraw, ImageFont
import cv2
import tifffile
from skimage import exposure
from numba import njit, jit

def loadingBar(count, total):
    sys.stdout.write("\r" + "Conversion progress:" + str(int(count)) + "/" + str(int(total)))
    
@njit #Gaussian adjust the image
def Gaussian_adjust(image):
    m = np.mean(image.ravel())
    s = np.std(image.ravel())
    g = np.exp(-0.5*((image-m)/s)**2)
    
    image = np.where(np.logical_and(g < 0.001, image > m), m + 3.5 * s, image)
                
    return image


@njit #normalize the image
def norm(im):
    return (im - np.min(im)) / (np.max(im) - np.min(im))

#save tif 
def saveTif(array, fileName):
    array_16bit = (array * 65535).astype(np.uint16)
    imsave(fileName, array_16bit)

@njit #round the numbers for the scale bar
def round_to_first_digit(number):
    power_of_10 = 10 ** int(np.log10(abs(number)))
    rounded_number = round(number / power_of_10) * power_of_10
    return rounded_number



#burn the scale bar onto the image
def write_text_to_tiff(image_path, text, position, font_path=None, font_size=20, font_color=(255, 255, 255)):
    # Load the image
    image = tifffile.imread(image_path)
    
    # Compute normalization factor
    factor = np.iinfo(image.dtype).max
    
    # Normalize the image
    image_norm = image.astype(np.float64) / factor
    
    # Convert to uint16
    image_norm_uint16 = (image_norm * np.iinfo(np.uint16).max).astype(np.uint16)
    
    # Create an ImageDraw object
    image_pil = Image.fromarray(image_norm_uint16)
    draw = ImageDraw.Draw(image_pil)

    # Load the font
    if font_path is not None:
        font = ImageFont.truetype(font_path, font_size)
    else:
        font = ImageFont.load_default()

    # Add the text to the image
    draw.text(position, text, font=font, fill=font_color)

    # Convert back to numpy array
    image_norm_uint16 = np.array(image_pil)
    
    # Convert back to float64
    image_norm = image_norm_uint16.astype(np.float64) / np.iinfo(np.uint16).max
    
    # Scale back to the original range
    image = (image_norm * factor).astype(image.dtype)
    
    tifffile.imwrite(image_path, image)

#make the folders to store the unconverted and converted images
def folderManager(folder, inExtension):
    outFolderWithBar = converted_images_path = os.path.join(folder, "convertedImagesWithBar") + "/"
    outFolderWithoutBar = converted_images_path = os.path.join(folder, "convertedImagesWithoutBar") + "/"
    inFolder = converted_images_path = os.path.join(folder, "dm4Images") + "/"
    if not os.path.exists(outFolderWithBar): os.makedirs(outFolderWithBar)
    if not os.path.exists(outFolderWithoutBar): os.makedirs(outFolderWithoutBar)
    if not os.path.exists(inFolder): os.makedirs(inFolder)
    return inFolder, outFolderWithBar, outFolderWithoutBar

#move the files in the general folder to unconverted
def copyImages(folder, inFolder):
    dm4_files = []
    for root, dirs, files in os.walk(folder):
        dm4_files.extend([os.path.join(root, file) for file in files if file.endswith('.dm4')])
        
    for source_file_path in dm4_files:
        file_name = os.path.basename(source_file_path)
        destination_file_path = os.path.join(inFolder, file_name)
        
        # Skip copying if source and destination are the same
        if source_file_path != destination_file_path:
            shutil.copy2(source_file_path, destination_file_path)



@njit
def add_scale_bar(normArray, pixelNumber, scaleBarPixels):
    scaleX1 = int(pixelNumber * .05)
    scaleX2 = scaleX1 + int(scaleBarPixels)
    scaleY1 = int(pixelNumber * .90)
    scaleY2 = int(pixelNumber * .9075)
    normArray[scaleY1:scaleY2, scaleX1:scaleX2] = 1
    return normArray


def convertImages(path, outFolderWithBar, outFolderWithoutBar, file):
    # Convert the image properly
    image = nc.read(path)
    imageArray = image["data"]
    
    # Check if the image data is complex
    if np.iscomplexobj(imageArray):
        # If the image data is complex, convert it to real numbers
        imageArray = np.absolute(imageArray)
    
    adjustArray = Gaussian_adjust(imageArray)
    normArray = norm(adjustArray)
    
    # Get the image size
    pixelSize = image["pixelSize"][0]
    pixelNumber = np.sqrt(np.size(image["data"]))
    imageWidth = pixelSize * pixelNumber
    
    scaleBarSize = round_to_first_digit(imageWidth/3) #this is the number of nm
    scaleBarPixels = scaleBarSize * (pixelNumber/imageWidth)
    
    # Create copy of array for images without scale bar
    normArrayWithoutBar = np.copy(normArray)
    
    # Add the scale bar using Numba-accelerated function
    normArray = add_scale_bar(normArray, pixelNumber, scaleBarPixels)

    # Name the new file
    fileName, fileExtension = os.path.splitext(file)
    fileNew = fileName + "_width=" + str(round(imageWidth,1))  + "nm"+ ".tif"
    
    # Save image with scale bar
    saveFileNameWithBar = outFolderWithBar + fileNew
    saveTif(normArray, saveFileNameWithBar)

    x = scaleX1 = int(pixelNumber * .05)
    y = scaleY2 = int(pixelNumber * .92)
    write_text_to_tiff(saveFileNameWithBar,str(scaleBarSize)+" nm",(x, y), font_path="/Library/Fonts/Arial.ttf", font_size=200, font_color=255)
    
    # Save image without scale bar
    saveFileNameWithoutBar = outFolderWithoutBar + fileNew
    saveTif(normArrayWithoutBar, saveFileNameWithoutBar)


def main(folder, inExtension):
    #Access folder manager
    inFolder, outFolderWithBar, outFolderWithoutBar = folderManager(folder, inExtension)
    
    #Copy the files in the general folder to unconverted
    copyImages(folder, inFolder)
    
    #Calculate the total file number
    totalFileNumber = len([f for f in os.listdir(inFolder) if os.path.isfile(os.path.join(inFolder, f))])
    
    #Loop over file numbers 
    for index, file in enumerate(os.listdir(inFolder)):
        path = os.path.join(inFolder, file)
        
        #Call functions to convert the images
        convertImages(path, outFolderWithBar, outFolderWithoutBar, file)

        loadingBar(index+1, totalFileNumber)
                

In [2]:
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230706/Cu3P 280stable"
#main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230706/Ni2P"
#main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230707/A4 - Cu3P 320 Hot injection - Dylan"
#main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230706/Cu3P - YB heat up"
#main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230707/B1 - InCuP - Yasin"
#main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230707/B1 - InCuP - Yasin"
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230717/Cu3P hot injection 280 stabilize"; main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230717/Cu3P hot injection 300 stabilize"; main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230717/Cu3P hot injection 320 stabilize"; main(folder, ".dm4")
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230720/Cu3P 280-260-wool for XW"; main(folder, ".dm4")
folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230720/Cu3P 280-260-wool for XW #2"; main(folder, ".dm4")

/var/folders/nb/xpgvp3fs1g7d95x_p77zlgqm0000gn/T/ipykernel_7512/4245518773.py:24: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  image = np.where(np.logical_and(g < 0.001, image > m), m + 3.5 * s, image)


Conversion progress:3/3

# Turn images into TIF

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt 
import os
import ncempy.io as nc
from skimage.io import imsave
import sys
import shutil
from PIL import Image, ImageDraw, ImageFont
import cv2
import tifffile
from skimage import exposure

In [1]:
def loadingBar(count, total):
    sys.stdout.write("\r" + "Conversion progress:" + str(int(count)) + "/" + str(int(total)))

def gaussian(x, m, s):
    return np.exp(-0.5*((x-m)/s)**2)

def Gaussian_adjust(image):
    im = image.copy()
    m = np.mean(im.ravel())
    s = np.std(im.ravel())
    g = gaussian(im, m, s)
    im[np.logical_and(g<0.001, im>m)] = m+3.5*s
    return im

def norm(im):
    return (im - np.min(im)) / (np.max(im) - np.min(im))

def saveTif(array, fileName):
    array_16bit = (array * 65535).astype(np.uint16)
    imsave(fileName, array_16bit)

def round_to_first_digit(number):
    power_of_10 = 10 ** int(np.log10(abs(number)))
    rounded_number = round(number / power_of_10) * power_of_10
    return rounded_number

def write_text_to_tiff(image_path, text, position, font_path=None, font_size=20, font_color=(255, 255, 255)):
    # Load the image
    image = tifffile.imread(image_path)
    
    # Compute normalization factor
    factor = np.iinfo(image.dtype).max
    
    # Normalize the image
    image_norm = image.astype(np.float64) / factor
    
    # Convert to uint16
    image_norm_uint16 = (image_norm * np.iinfo(np.uint16).max).astype(np.uint16)
    
    # Create an ImageDraw object
    image_pil = Image.fromarray(image_norm_uint16)
    draw = ImageDraw.Draw(image_pil)

    # Load the font
    if font_path is not None:
        font = ImageFont.truetype(font_path, font_size)
    else:
        font = ImageFont.load_default()

    # Add the text to the image
    draw.text(position, text, font=font, fill=font_color)

    # Convert back to numpy array
    image_norm_uint16 = np.array(image_pil)
    
    # Convert back to float64
    image_norm = image_norm_uint16.astype(np.float64) / np.iinfo(np.uint16).max
    
    # Scale back to the original range
    image = (image_norm * factor).astype(image.dtype)
    
    tifffile.imwrite(image_path, image)


def convertImages(folder, inExtension):
    #make the folders to store the unconverted and converted images
    outFolderWithBar = converted_images_path = os.path.join(folder, "convertedImagesWithBar") + "/"
    outFolderWithoutBar = converted_images_path = os.path.join(folder, "convertedImagesWithoutBar") + "/"
    inFolder = converted_images_path = os.path.join(folder, "dm4Images") + "/"
    if not os.path.exists(outFolderWithBar): os.makedirs(outFolderWithBar)
    if not os.path.exists(outFolderWithoutBar): os.makedirs(outFolderWithoutBar)
    if not os.path.exists(inFolder): os.makedirs(inFolder)
    
    #move the files in the general folder to unconverted
    dm4_files = []
    for root, dirs, files in os.walk(folder):
        dm4_files.extend([os.path.join(root, file) for file in files if file.endswith('.dm4')])
        
    for source_file_path in dm4_files:
        file_name = os.path.basename(source_file_path)
        destination_file_path = os.path.join(inFolder, file_name)
        shutil.move(source_file_path, destination_file_path)
        
    totalFileNumber = len([f for f in os.listdir(inFolder) if os.path.isfile(os.path.join(inFolder, f))])
    for index, file in enumerate(os.listdir(inFolder)):
        if file.endswith(inExtension):
            path = os.path.join(inFolder, file)
            if os.path.isfile(path):
                
                #convert the image properly
                image = nc.read(path)
                imageArray = image["data"]
                adjustArray = Gaussian_adjust(imageArray)
                normArray = norm(adjustArray)
                
                #get the image size
                pixelSize = image["pixelSize"][0]
                pixelNumber = np.sqrt(np.size(image["data"]))
                imageWidth = pixelSize * pixelNumber
                
                scaleBarSize = round_to_first_digit(imageWidth/3) #this is the number of nm
                scaleBarPixels = scaleBarSize * (pixelNumber/imageWidth)
                
                #create copy of array for images without scale bar
                normArrayWithoutBar = np.copy(normArray)
                
                #add the scale bar
                scaleX1 = int(pixelNumber * .05)
                scaleX2 = scaleX1+int(scaleBarPixels)
                scaleY1 = int(pixelNumber * .90)
                scaleY2 = int(pixelNumber * .9075)
                normArray[scaleY1:scaleY2, scaleX1:scaleX2] = 1
                
                #name the new file
                fileName, fileExtension = os.path.splitext(file)
                fileNew = fileName + "_width=" + str(round(imageWidth,1))  + "nm"+ ".tif"
                
                #Save image with scale bar
                saveFileNameWithBar = outFolderWithBar + fileNew
                saveTif(normArray, saveFileNameWithBar)

                x = scaleX1 = int(pixelNumber * .05)
                y = scaleY2 = int(pixelNumber * .92)
                write_text_to_tiff(saveFileNameWithBar,str(scaleBarSize)+" nm",(x, y), font_path="/Library/Fonts/Arial.ttf", font_size=200, font_color=255)
                
                #Save image without scale bar
                saveFileNameWithoutBar = outFolderWithoutBar + fileNew
                saveTif(normArrayWithoutBar, saveFileNameWithoutBar)
                
                #enhance_contrast(saveFileName, contrast_method="auto", low_percentile=5, high_percentile=95, alpha=2.0, beta=-50)
                
        loadingBar(index+1, totalFileNumber)


In [6]:
#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230701/A4 - WB"
#inExtension = ".dm4"
#convertImages(folder, inExtension)

#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230701/B4 - WB"
#inExtension = ".dm4"
#convertImages(folder, inExtension)

#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230701/E2 - YB"
#inExtension = ".dm4"
#convertImages(folder, inExtension)

#folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230701/E3 - WB"
#inExtension = ".dm4"
#convertImages(folder, inExtension)

folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/230703/"
inExtension = ".dm4"
convertImages(folder, inExtension)

folder = "/Users/yasinbadawy/Library/CloudStorage/Box-Box/TEM Images - Yasin/2307041/"
inExtension = ".dm4"
convertImages(folder, inExtension)

Conversion progress:87/87